In [96]:
import requests
import os
url = 'http://127.0.0.1:'+str(2902)+'/recommend'

import time
starttime = time.time()
x = requests.get(url, params={"uuid":"53ee0a27-c4c4-461b-8285-adebdc6df5e3"})
print("Time elapsed: ", round(time.time() - starttime, 2))
print(x.text)

Time elapsed:  4.28
{"size":0}


In [1]:
from pymongo_getdb import get_database
db = get_database()

In [44]:
import time
def get_attended_and_available(db, uuid:str):
    past_event_ids = []
    for event in db["attendances"].find({'user':uuid}):
        past_event_ids.append(event["uuid"])
    past_events = []
    for past_event_id in past_event_ids:
        for event in db["events"].find({"uuid": past_event_id}):
            past_events.append(dict(event))
    available_events = [event for event in db["events"].find({"date": {'$gt': time.time() * 1000}})]
    return past_events, available_events

In [50]:
from utils import get_scores
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('average_word_embeddings_glove.6B.300d')

attended, available = get_attended_and_available(db, "53ee0a27-c4c4-461b-8285-adebdc6df5e3")
for i in attended:
    i["zip_code"] = i["zip"]
    del i["zip"]
for i in available:
    i["zip_code"] = i["zip"]
    del i["zip"]

scores = get_scores(model, attended, available)

In [51]:
scores

defaultdict(<function utils.get_scores.<locals>.<lambda>()>,
            {'9d9ec8fe-85d8-40e9-9dc0-88ef963ba7e4': 14.354244239628315,
             '8ca87f53-9f82-4564-8f76-637c5fee3b97': 22.74951690249145,
             '52ce2ea9-622a-4e25-903b-4ed057127abb': 12.318105518817902,
             '1410e4b2-4bd8-4611-a769-e212295acf7c': 16.397451780736446,
             'cb4efb19-b1f9-4ce3-978f-03d1b0fdd60e': 10.778715774416924})

In [38]:
import time
for event in db["events"].find({"date": {'$gt': time.time() * 1000}}):
    print(event)

{'_id': ObjectId('64781b02944e8b635368c59b'), 'title': 'test', 'description': 'descritpion', 'date': 11076307200000.0, 'zip': 91362, 'type': 3, 'organizer': 'Charles Qin', 'uuid': '9d9ec8fe-85d8-40e9-9dc0-88ef963ba7e4', 'likes': nan}
{'_id': ObjectId('64782612bd542abbc82e23fd'), 'title': 'Dislike button', 'description': 'i wanna dislike konst bday party', 'date': 14214441600000.0, 'zip': 91362, 'type': 5, 'organizer': 'Charles Qin', 'uuid': '8ca87f53-9f82-4564-8f76-637c5fee3b97', 'likes': 4}
{'_id': ObjectId('647856a7fbbbc12ed4466f66'), 'title': 'asdfad', 'description': 'fasdfas', 'date': 4803753600000.0, 'zip': 91362, 'type': 1, 'organizer': 'Charles Qin', 'uuid': '52ce2ea9-622a-4e25-903b-4ed057127abb', 'likes': 1, 'liked': 0}
{'_id': ObjectId('647856befbbbc12ed4466f67'), 'title': 'testing', 'description': 'testest', 'date': 11427350400000.0, 'zip': 91362, 'type': 1, 'organizer': 'Charles Qin', 'uuid': '1410e4b2-4bd8-4611-a769-e212295acf7c', 'likes': 2, 'liked': 0}
{'_id': ObjectId('6

In [25]:
get_past_events('53ee0a27-c4c4-461b-8285-adebdc6df5e3')

[{'_id': ObjectId('647817c1c7858d9bd4765bfe'),
  'title': 'Test Event 1',
  'description': 'This is a test event',
  'date': 1685491200000.0,
  'zip': 94588,
  'type': 1,
  'organizer': 'SEAN TANG',
  'uuid': 'e755fec1-379c-4030-9d93-34bdd29aed7b',
  'likes': 3},
 {'_id': ObjectId('647824ecef20b8f047a88652'),
  'title': 'Guys also plz fix the UI 😩',
  'description': "The form for posting events doesn't look too nice rn :(",
  'date': 1685491200000.0,
  'zip': 90024,
  'type': 1,
  'organizer': 'Konstantin Tzantchev',
  'uuid': '3068a791-3ab4-440c-b4e8-50f9e634df0d',
  'likes': 5},
 {'_id': ObjectId('647817f9c7858d9bd4765c00'),
  'title': "Konstantine' B-day",
  'description': 'Please dont let me have 0 attendance plz',
  'date': 1685491200000.0,
  'zip': 94588,
  'type': 5,
  'organizer': 'SEAN TANG',
  'uuid': '725ee160-88c8-4de3-bff7-257f1e973012',
  'likes': 3},
 {'_id': ObjectId('64782612bd542abbc82e23fd'),
  'title': 'Dislike button',
  'description': 'i wanna dislike konst bday p